In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10

In [2]:
samp=pd.read_csv('sample_submission.csv')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.loc[:, 'sample'] = 'train'
test.loc[:, 'sample'] = 'test'

In [4]:
test = test.merge(samp, on='_id')
df = test.append(train).reset_index(drop=True)
df.head()

,_id,age,campaign,cons.conf.idx,cons.price.idx,contact,day_of_week,default,duration,education,...,job,loan,marital,month,nr.employed,pdays,poutcome,previous,sample,target
0,66810d8e6bf2b41c880a7bc6c8a1e295,57,1,-46.2,92.893,cellular,mon,no,371,high.school,...,technician,yes,married,may,5099.1,999,failure,1,test,0
1,ccac3879652b08cb8b44c1920fd93afa,55,2,-36.4,93.994,telephone,thu,unknown,285,unknown,...,unknown,no,married,may,5191.0,999,nonexistent,0,test,0
2,fcccab4d7a76f70647f015f2c84c2af8,33,1,-46.2,92.893,cellular,fri,no,52,basic.9y,...,blue-collar,no,married,may,5099.1,999,failure,1,test,0
3,ed8399278c30678dab739045fa12b440,36,4,-41.8,94.465,telephone,fri,no,355,high.school,...,admin.,no,married,jun,5228.1,999,nonexistent,0,test,0
4,1d4d62ac5cabcb48bac7112813f290cb,27,2,-42.7,93.918,cellular,fri,no,189,high.school,...,housemaid,no,married,jul,5228.1,999,nonexistent,0,test,0


In [5]:
dtt = df.copy()
y = dtt[['target']]
dtt = dtt.drop(('target'), axis=1)
dtt = dtt.drop(('_id'), axis=1)
sm = dtt[['sample']]
dtt = dtt.drop(('sample'), axis=1)

In [6]:
def preproc_data(df_input):
    
    train = df_input.copy()
    
    data_describe1 = train['default'].describe()
    train['default'].replace(['unknown'], [data_describe1['top']], inplace=True)
           
    data_describe2 = train['housing'].describe()
    train['housing'].replace(['unknown'], [data_describe2['top']], inplace=True)
    
    data_describe3 = train['loan'].describe()
    train['loan'].replace(['unknown'], [data_describe3['top']], inplace=True)
    
    data_describe4 = train['marital'].describe()
    train['marital'].replace(['unknown'], [data_describe4['top']], inplace=True)
    
    data_describe5 = train['education'].describe()
    train['education'].replace(['unknown'], [data_describe5['top']], inplace=True)
    
    data_describe6 = train['job'].describe()
    train['job'].replace(['unknown'], [data_describe6['top']], inplace=True)
    
 
    train['education'].replace(["basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree",], [0,1,2,3,4,5,6], inplace=True)
 
    
    categorical_columns = [c for c in df_input.columns if train[c].dtype.name == 'object']
    numerical_columns   = [c for c in df_input.columns if train[c].dtype.name != 'object']

    for i in categorical_columns:
        print (train[i].unique())
    
    train = train.fillna(train.median(axis=0), axis=0)

    data_describe = train.describe(include=[object])
    for c in categorical_columns:
        train[c] = train[c].fillna(data_describe[c]['top'])
    
    train.describe(include=[object])    

    binary_columns = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
    nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]

    for c in binary_columns:
        top = data_describe[c]['top']
        top_items = train[c] == top
        train.loc[top_items, c] = 0
        train.loc[np.logical_not(top_items), c] = 1

    train[binary_columns].describe()

    data_nonbinary = pd.get_dummies(train[nonbinary_columns])


    data_numerical = train[numerical_columns]
    data_numerical

    data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
    data_numerical.describe()

    data = pd.concat((data_numerical, train[binary_columns], data_nonbinary), axis=1)
    data = pd.DataFrame(data, dtype=float)

    return data

In [7]:
dat = preproc_data(dtt)

['cellular' 'telephone']
['mon' 'thu' 'fri' 'wed' 'tue']
['no' 'yes']
['no' 'yes']
['technician' 'admin.' 'blue-collar' 'housemaid' 'retired' 'services'
 'entrepreneur' 'unemployed' 'management' 'self-employed' 'student']
['yes' 'no']
['married' 'divorced' 'single']
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'apr' 'mar' 'sep' 'dec']
['failure' 'nonexistent' 'success']


In [8]:
dat = dat.drop(dat.columns[[15,20,31,34,44]], axis=1)

In [9]:
dat.head()

,age,campaign,cons.conf.idx,cons.price.idx,duration,education,emp.var.rate,euribor3m,nr.employed,pdays,...,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_nonexistent,poutcome_success
0,1.628973,-0.565915,-1.231019,-1.179366,0.434724,-0.338822,-1.197921,-1.338923,-0.940270,0.195412,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.437058,-0.204906,0.886436,0.722714,0.103036,1.102653,0.648084,0.714181,0.331676,0.195412,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-0.674013,-0.565915,-1.231019,-1.179366,-0.795609,-0.819314,-1.197921,-1.330851,-0.940270,0.195412,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.386140,0.517112,-0.280325,1.536410,0.373015,-0.338822,0.839050,0.775872,0.845160,0.195412,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1.249760,-0.204906,-0.474785,0.591417,-0.267222,-0.338822,0.839050,0.773566,0.845160,0.195412,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dat, y, test_size=0.3, random_state=123)

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 

In [15]:
for i in range(2,10):
    algo = KNeighborsClassifier(n_neighbors=i).fit(X_train, y_train)
    print("Кол-во соседей: {}. Правильность на трейне: {:.3f}".format(i, algo.score(X_train, y_train)))
    print("Кол-во соседей: {}. Правильность на тесте: {:.3f}".format(i, algo.score(X_test, y_test)))

/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Кол-во соседей: 2. Правильность на трейне: 0.951
Кол-во соседей: 2. Правильность на тесте: 0.928
Кол-во соседей: 3. Правильность на трейне: 0.951
Кол-во соседей: 3. Правильность на тесте: 0.920
Кол-во соседей: 4. Правильность на трейне: 0.943
Кол-во соседей: 4. Правильность на тесте: 0.928
Кол-во соседей: 5. Правильность на трейне: 0.943
Кол-во соседей: 5. Правильность на тесте: 0.924
Кол-во соседей: 6. Правильность на трейне: 0.940
Кол-во соседей: 6. Правильность на тесте: 0.930
Кол-во соседей: 7. Правильность на трейне: 0.941
Кол-во соседей: 7. Правильность на тесте: 0.928
Кол-во соседей: 8. Правильность на трейне: 0.938
Кол-во соседей: 8. Правильность на тесте: 0.930
Кол-во соседей: 9. Правильность на трейне: 0.939
Кол-во соседей: 9. Правильность на тесте: 0.929


In [12]:
for i in range(1,10):
    algo = DecisionTreeClassifier(max_depth=i).fit(X_train, y_train)
    print("Кол-во соседей: {}. Правильность на трейне: {:.4f}".format(i, algo.score(X_train, y_train)))
    print("Кол-во соседей: {}. Правильность на тесте: {:.4f}".format(i, algo.score(X_test, y_test)))

Кол-во соседей: 1. Правильность на трейне: 0.9324
Кол-во соседей: 1. Правильность на тесте: 0.9323
Кол-во соседей: 2. Правильность на трейне: 0.9324
Кол-во соседей: 2. Правильность на тесте: 0.9323
Кол-во соседей: 3. Правильность на трейне: 0.9324
Кол-во соседей: 3. Правильность на тесте: 0.9323
Кол-во соседей: 4. Правильность на трейне: 0.9328
Кол-во соседей: 4. Правильность на тесте: 0.9323
Кол-во соседей: 5. Правильность на трейне: 0.9338
Кол-во соседей: 5. Правильность на тесте: 0.9323
Кол-во соседей: 6. Правильность на трейне: 0.9353
Кол-во соседей: 6. Правильность на тесте: 0.9315
Кол-во соседей: 7. Правильность на трейне: 0.9368
Кол-во соседей: 7. Правильность на тесте: 0.9294
Кол-во соседей: 8. Правильность на трейне: 0.9393
Кол-во соседей: 8. Правильность на тесте: 0.9270
Кол-во соседей: 9. Правильность на трейне: 0.9434
Кол-во соседей: 9. Правильность на тесте: 0.9235


In [17]:
for i in [0.1, 0.001,0.005, 10, 100]:
    algo = LogisticRegression(C=i).fit(X_train, y_train)
    print("Кол-во соседей: {}. Правильность на трейне: {:.5f}".format(i, algo.score(X_train, y_train)))
    print("Кол-во соседей: {}. Правильность на тесте: {:.5f}".format(i, algo.score(X_test, y_test)))

/srv/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Кол-во соседей: 0.1. Правильность на трейне: 0.93046
Кол-во соседей: 0.1. Правильность на тесте: 0.93243
Кол-во соседей: 0.001. Правильность на трейне: 0.93143
Кол-во соседей: 0.001. Правильность на тесте: 0.93251
Кол-во соседей: 0.005. Правильность на трейне: 0.93122
Кол-во соседей: 0.005. Правильность на тесте: 0.93162
Кол-во соседей: 10. Правильность на трейне: 0.93004
Кол-во соседей: 10. Правильность на тесте: 0.93178
Кол-во соседей: 100. Правильность на трейне: 0.92997
Кол-во соседей: 100. Правильность на тесте: 0.93178


In [18]:
#ОТВЕТ
#Оптимальное решение DecisionTreeClassifier(max_depth=5):
print("Глубина: {}. Правильность на трейне: {:.5f}".format(5, DecisionTreeClassifier(max_depth=5).fit(X_train, y_train).score(X_train, y_train)))
print("Глубина: {}. Правильность на тесте: {:.5f}".format(5, DecisionTreeClassifier(max_depth=5).fit(X_train, y_train).score(X_test, y_test)))

Глубина: 5. Правильность на трейне: 0.93379
Глубина: 5. Правильность на тесте: 0.93227


In [ ]:
#Ответ: 0.93227